# Chatbot para Análisis de A/B Testing

Este notebook implementa un chatbot que puede leer y analizar datos de experimentos A/B testing desde archivos CSV/Excel y responder preguntas sobre los resultados.

## 1. Configuración del Entorno

### Dependencias Necesarias
```bash
pip install pandas numpy scipy openai python-dotenv openpyxl
```

### Variables de Entorno
Crea un archivo `.env` en el mismo directorio con:
```
OPENAI_API_KEY=tu_api_key_aqui
```

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
from typing import Dict, List, Any

# Cargar variables de entorno
load_dotenv()

# Verificar que la API key esté configurada
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("❌ Error: OPENAI_API_KEY no encontrada en el archivo .env")
    print("💡 Solución: Crea un archivo .env con tu API key:")
    print("   OPENAI_API_KEY=tu_api_key_aqui")
    client = None
else:
    try:
        client = OpenAI(api_key=api_key)
        print("✅ Cliente OpenAI inicializado correctamente")
    except Exception as e:
        print(f"❌ Error al inicializar OpenAI: {e}")
        client = None

✅ Cliente OpenAI inicializado correctamente


## 2. Clase Principal del Chatbot

In [ ]:
class ABTestingChatbot:
    def __init__(self):
        self.detailed_data = None
        self.analysis_cache = {}
        
        # Inicializar cliente OpenAI
        api_key = os.getenv('OPENAI_API_KEY')
        if api_key:
            self.client = OpenAI(api_key=api_key)
        else:
            self.client = None
            print("⚠️ Warning: No se pudo inicializar OpenAI. Configura OPENAI_API_KEY en .env")
        
    def load_data(self, detailed_file: str):
        """
        Carga solo los datos detallados del experimento
        """
        if detailed_file.endswith('.csv'):
            self.detailed_data = pd.read_csv(detailed_file)
        elif detailed_file.endswith(('.xlsx', '.xls')):
            self.detailed_data = pd.read_excel(detailed_file)
        
        print(f"✅ Datos detallados cargados:")
        print(f"   - {len(self.detailed_data)} filas")
        print(f"   - Experimentos: {', '.join(self.detailed_data['experimento'].unique())}")
    
    def generate_context(self, question: str) -> str:
        """
        Genera contexto relevante usando solo datos detallados
        """
        if self.detailed_data is None:
            return "No hay datos cargados"
        
        context_parts = []
        
        # Información básica de los datos
        context_parts.append("DATOS DETALLADOS DEL EXPERIMENTO:")
        context_parts.append(f"Total de filas: {len(self.detailed_data)}")
        context_parts.append(f"Experimentos: {', '.join(self.detailed_data['experimento'].unique())}")
        context_parts.append(f"Columnas: {', '.join(self.detailed_data.columns)}")
        
        # Incluir todos los datos detallados para que el LLM pueda hacer cálculos
        context_parts.append("\nDATOS COMPLETOS:")
        context_parts.append(self.detailed_data.to_string(index=False))
        
        return "\n".join(context_parts)
    
    def chat(self, question: str) -> str:
        """
        Procesa una pregunta y devuelve respuesta del chatbot
        """
        if self.detailed_data is None:
            return "❌ Primero debes cargar los datos usando load_data()"
        
        if self.client is None:
            return "❌ Cliente OpenAI no inicializado. Verifica tu OPENAI_API_KEY en el archivo .env"
        
        # Generar contexto
        context = self.generate_context(question)
        
        # Preparar prompt
        system_prompt = """
        Eres un analista experto en A/B testing. Tu trabajo es analizar datos detallados de experimentos y responder preguntas calculando métricas desde los datos granulares.
        
        IMPORTANTE:
        - Debes hacer todos los cálculos basándote en los datos detallados proporcionados
        - Para totales: suma todas las filas relevantes
        - Para promedios: calcula desde los datos individuales
        - Para comparaciones: agrupa y compara los datos según la pregunta
        - Muestra tus cálculos cuando sea relevante

        NUNCA hagas sumas manuales. En su lugar:
        1. Identifica qué experimento necesitas analizar
        2. Filtra solo esas filas
        3. Describe el proceso: "Sumando usuarios de X filas del experimento Y"
        4. Da el resultado final
        
        Características de tus respuestas:
        - Basadas en cálculos precisos de los datos detallados
        - Incluyen números específicos y cómo los calculaste
        - Identifican patrones en los datos granulares
        - Son claras y muestran el razonamiento
        
        Si necesitas hacer algún cálculo complejo, explica paso a paso cómo lo obtuviste.
        """
        
        user_prompt = f"""
        PREGUNTA: {question}
        
        CONTEXTO DE DATOS:
        {context}
        
        Responde la pregunta calculando desde los datos detallados proporcionados. Muestra tu proceso de cálculo cuando sea relevante.
        """
        
        try:
            response = self.client.chat.completions.create(
                model="GPT-4.5",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=1000,
                temperature=0.1
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"❌ Error al consultar OpenAI: {str(e)}"
    
    def show_data_preview(self):
        """
        Muestra vista previa de los datos cargados
        """
        if self.detailed_data is not None:
            print("📋 DATOS DETALLADOS:")
            print(self.detailed_data.head(10))
            print(f"\nShape: {self.detailed_data.shape}")
            print(f"Columnas: {list(self.detailed_data.columns)}")
            
            print(f"\n📊 Resumen por experimento:")
            for exp in self.detailed_data['experimento'].unique():
                exp_data = self.detailed_data[self.detailed_data['experimento'] == exp]
                print(f"   {exp}: {len(exp_data)} tiendas, {exp_data['usuarios'].sum()} usuarios")
        else:
            print("❌ No hay datos cargados")

## 3. Ejemplo de Uso

### Crear datos de ejemplo para pruebas

In [8]:
# Crear datos de ejemplo
np.random.seed(42)

# Solo crear datos detallados por tienda (sin resumen)
tiendas_detalle = []
experimentos = ['Control', 'Experimento_A', 'Experimento_B', 'Experimento_C']
base_conversion_rates = [8.5, 9.4, 10.2, 11.9]

for exp, base_conversion in zip(experimentos, base_conversion_rates):
    for i in range(50):  # 50 tiendas por experimento
        tiendas_detalle.append({
            'experimento': exp,
            'tienda_id': f'T_{exp}_{i+1:03d}',
            'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste']),
            'tipo_tienda': np.random.choice(['Mall', 'Street', 'Outlet']),
            'usuarios': np.random.randint(150, 300),
            'conversiones': None,
            'revenue': None,
            'conversion_rate': np.random.normal(base_conversion, 2.5)
        })

tiendas_df = pd.DataFrame(tiendas_detalle)
tiendas_df['conversiones'] = (tiendas_df['usuarios'] * tiendas_df['conversion_rate'] / 100).round().astype(int)
tiendas_df['revenue'] = tiendas_df['conversiones'] * np.random.uniform(15, 25, len(tiendas_df))
tiendas_df['conversion_rate'] = tiendas_df['conversion_rate'].round(2)

# Guardar solo el archivo de datos detallados
tiendas_df.to_csv('tiendas_detalle.csv', index=False)

print("✅ Archivo de datos detallados creado:")
print("   - tiendas_detalle.csv")
print(f"\n📊 Datos generados:")
print(f"   - Total filas: {len(tiendas_df)}")
print(f"   - Experimentos: {tiendas_df['experimento'].unique()}")
print(f"   - Tiendas por experimento: {len(tiendas_df) // len(experimentos)}")

# Mostrar estadísticas por experimento
print(f"\n📈 Estadísticas por experimento:")
for exp in experimentos:
    exp_data = tiendas_df[tiendas_df['experimento'] == exp]
    print(f"   {exp}: {exp_data['usuarios'].sum()} usuarios, {exp_data['conversiones'].sum()} conversiones")

✅ Archivo de datos detallados creado:
   - tiendas_detalle.csv

📊 Datos generados:
   - Total filas: 200
   - Experimentos: ['Control' 'Experimento_A' 'Experimento_B' 'Experimento_C']
   - Tiendas por experimento: 50

📈 Estadísticas por experimento:
   Control: 11196 usuarios, 986 conversiones
   Experimento_A: 11405 usuarios, 1073 conversiones
   Experimento_B: 11900 usuarios, 1202 conversiones
   Experimento_C: 11199 usuarios, 1304 conversiones


### Inicializar el Chatbot

In [3]:
# Crear instancia del chatbot
chatbot = ABTestingChatbot()

# Cargar solo datos detallados
chatbot.load_data('tiendas_detalle.csv')

# Mostrar preview
chatbot.show_data_preview()

✅ Datos detallados cargados:
   - 200 filas
   - Experimentos: Control, Experimento_A, Experimento_B, Experimento_C
📋 DATOS DETALLADOS:
  experimento      tienda_id region tipo_tienda  usuarios  conversiones  \
0     Control  T_Control_001   Este        Mall       164            17   
1     Control  T_Control_002   Este      Street       224            28   
2     Control  T_Control_003   Este        Mall       249            24   
3     Control  T_Control_004  Norte      Street       237            15   
4     Control  T_Control_005  Oeste      Street       187            23   
5     Control  T_Control_006  Norte        Mall       198            24   
6     Control  T_Control_007   Este      Outlet       164            20   
7     Control  T_Control_008   Este      Outlet       213            17   
8     Control  T_Control_009  Norte      Outlet       200            14   
9     Control  T_Control_010    Sur        Mall       209            28   

      revenue  conversion_rate  
0  31

## 4. Ejemplos de Preguntas

In [6]:
# Pregunta 1: ¿Cuál fue el mejor experimento?
pregunta_1 = "¿Cuál fue el mejor experimento y por qué?"
print(f"🤖 Pregunta: {pregunta_1}")
print(f"💬 Respuesta: {chatbot.chat(pregunta_1)}")
print("\n" + "="*80 + "\n")

🤖 Pregunta: ¿Cuál fue el mejor experimento y por qué?
💬 Respuesta: El mejor experimento fue Experimento_C. Este experimento tuvo un total de 1180 conversiones, generando un revenue total de 189,000 unidades monetarias. Además, tuvo la tasa de conversión más alta de 11.9% y el revenue por usuario más alto de 19.1 unidades monetarias. 

Comparado con el grupo de control, Experimento_C tuvo un ROI significativamente mayor de 52.8, lo que indica un rendimiento mucho mejor en términos de retorno de la inversión.

Por lo tanto, basándonos en los datos proporcionados, Experimento_C fue el mejor en términos de métricas clave y retorno de la inversión.




In [7]:
# Pregunta 2: Análisis de significancia
pregunta_2 = "¿Cuáles experimentos son estadísticamente significativos y qué significa esto?"
print(f"🤖 Pregunta: {pregunta_2}")
print(f"💬 Respuesta: {chatbot.chat(pregunta_2)}")
print("\n" + "="*80 + "\n")

🤖 Pregunta: ¿Cuáles experimentos son estadísticamente significativos y qué significa esto?
💬 Respuesta: Basándome en los datos proporcionados, los experimentos estadísticamente significativos son Experimento_A, Experimento_B y Experimento_C. Esto significa que los resultados obtenidos en estos experimentos no se deben al azar, sino que son resultado de las variaciones introducidas en comparación con el grupo de control de manera significativa.

- Experimento_A tiene un p-value de 0.0230, lo que indica que hay una diferencia significativa en comparación con el grupo de control.
- Experimento_B tiene un p-value de 0.0010, mostrando una diferencia aún más significativa.
- Experimento_C es el más significativo con un p-value de 0.0001, lo que sugiere una diferencia altamente significativa.

Estos resultados indican que los Experimentos A, B y C han generado mejoras significativas en conversiones, ingresos totales, tasas de conversión, ingresos por usuario y retorno de inversión en comparac

In [8]:
# Pregunta 3: Análisis por tienda/región
pregunta_3 = "¿Hay diferencias en el performance por región o tipo de tienda?"
print(f"🤖 Pregunta: {pregunta_3}")
print(f"💬 Respuesta: {chatbot.chat(pregunta_3)}")
print("\n" + "="*80 + "\n")

🤖 Pregunta: ¿Hay diferencias en el performance por región o tipo de tienda?
💬 Respuesta: Para analizar si hay diferencias en el performance por región o tipo de tienda, podemos revisar los datos detallados de usuarios, conversiones, revenue y conversion rate por región y tipo de tienda en cada experimento.

### Por Región:
- **Este:** Experimento_C tiene el mejor desempeño en esta región, con una tasa de conversión de 11.9% y un revenue por usuario de 19.1.
- **Norte:** Experimento_C también destaca en esta región, con una tasa de conversión de 11.9% y un revenue por usuario de 19.1.
- **Oeste:** No hay datos detallados para el Oeste, por lo que no podemos comparar el desempeño en esta región.
- **Sur:** No hay datos detallados para el Sur, por lo que no podemos comparar el desempeño en esta región.

### Por Tipo de Tienda:
- **Mall:** Experimento_C muestra un buen desempeño en este tipo de tienda, con una tasa de conversión de 11.9% y un revenue por usuario de 19.1.
- **Street:** Expe

In [9]:
# Pregunta 4: Recomendaciones
pregunta_4 = "¿Qué experimento recomendarías implementar y por qué?"
print(f"🤖 Pregunta: {pregunta_4}")
print(f"💬 Respuesta: {chatbot.chat(pregunta_4)}")
print("\n" + "="*80 + "\n")

🤖 Pregunta: ¿Qué experimento recomendarías implementar y por qué?
💬 Respuesta: Basándome en los datos proporcionados, recomendaría implementar un nuevo experimento que combine las estrategias más exitosas de los Experimentos B y C. 

El Experimento B mostró un aumento significativo en conversiones, revenue total, conversion rate, revenue por usuario y ROI en comparación con el Control. Por otro lado, el Experimento C tuvo resultados aún más impresionantes en todas estas métricas, superando al Experimento B.

Al combinar elementos clave de ambos experimentos, como posiblemente la estrategia de marketing utilizada en el Experimento B con la optimización de la experiencia de usuario del Experimento C, es probable que se logren resultados aún más impactantes en términos de conversiones, ingresos y retorno de inversión.

Sería importante diseñar este nuevo experimento de manera cuidadosa, manteniendo un enfoque claro en las métricas clave que han demostrado ser significativas en experimento

## 5. Interfaz Interactiva

In [4]:
def chat_interactivo():
    """
    Función para chat interactivo
    """
    print("🤖 Chatbot de A/B Testing iniciado!")
    print("💡 Escribe 'salir' para terminar\n")
    
    while True:
        pregunta = input("🤔 Tu pregunta: ")
        
        if pregunta.lower() in ['salir', 'exit', 'quit']:
            print("👋 ¡Hasta luego!")
            break
        
        if pregunta.strip() == "":
            continue
            
        print(f"\n💬 Respuesta: {chatbot.chat(pregunta)}")
        print("\n" + "-"*50 + "\n")

# Descomentar la siguiente línea para usar el chat interactivo
chat_interactivo()

🤖 Chatbot de A/B Testing iniciado!
💡 Escribe 'salir' para terminar


💬 Respuesta: ❌ Error al consultar OpenAI: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}

--------------------------------------------------

👋 ¡Hasta luego!


## 6. Próximos Pasos

### Mejoras Posibles:
1. **Análisis más sofisticados**: Intervalos de confianza, power analysis
2. **Visualizaciones**: Gráficos automáticos con matplotlib/plotly
3. **Memoria conversacional**: Recordar preguntas anteriores
4. **Validación de datos**: Detección automática de anomalías
5. **Export de insights**: Generar reportes automáticos

### Para Usar con Tus Datos:
1. Reemplaza `experimentos_resumen.csv` y `tiendas_detalle.csv` con tus archivos
2. Ajusta los nombres de columnas en la clase si es necesario
3. Configura tu `OPENAI_API_KEY` en el archivo `.env`